In [10]:
from pymongo import MongoClient
# Connect to the mongo local database
connection = MongoClient()
db = connection.bsa_files
collection = db.bsa_files

In [12]:
print(connection)
print
print(db)
print
print(connection.database_names)
print
print(collection)
print
print(db.collection_names)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'bsa_files')

<bound method MongoClient.database_names of MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)>

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'bsa_files'), u'bsa_files')

<bound method Database.collection_names of Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'bsa_files')>


In [1]:
import xlsxwriter
import pickle

In [2]:
working_uid_list =  pickle.load(open('working_uid_list.pickle','r'))
working_hash_dict = pickle.load(open('working_hash_dict.pickle','r'))
content_hash_dict = pickle.load(open('content_hash_dict.pickle','r'))
unique_content_hash_list = pickle.load(open('unique_content_hash_list.pickle','r'))

#Validate
print(len(working_uid_list))
print(len(working_hash_dict))
print(len(content_hash_dict))
print(len(unique_content_hash_list))

36549
5546
4221
4221


In [4]:
export_columns = ['content_hash','bin_hash','doc_name','primary_url','alternate_url','pages',
                 'title','creation_date','producer','creator','last_modified','author']

In [50]:
#Converts the typical output for a PyPDF2 Metadata Date into a useable format. 
def convertPyPDFDate(origDate):
    if len(origDate) == 23:
        year = origDate[2:6]
        mon = origDate[6:8]
        day = origDate[8:10]
        return (year+"-"+mon+"-"+day)
    return origDate

def get_export_row(bin_hash):
    doc_name = None
    primary_url = None
    alternate_url = None
    pages = None
    title = None
    creation_date = None
    producer = None
    creator = None
    last_modified = None
    author = None
    
    alt_doc_name = None
    alt_primary_url = None
    alt_pages = None
    alt_title = None
    alt_creation_date = None
    alt_producer = None
    alt_creator = None
    alt_last_modified = None
    alt_author = None
    
    cursor = collection.find({"Hash":bin_hash})
    content_hash = cursor[0]['Content_Hash']
    binary_hash = bin_hash
    
    for document in cursor:
        if document['Domain']== u'www.scouting.org':
            doc_name = document['File_Name']
            primary_url = document['URL']
            pages = document['Pages']
            
            if '/Title' in document["Metadata"]:
                title = document["Metadata"]["/Title"]
            if '/CreationDate' in document["Metadata"]:
                creation_date = convertPyPDFDate(document["Metadata"]["/CreationDate"])
            if '/Producer' in document["Metadata"]:
                producer = document["Metadata"]["/Producer"]
            if '/Creator' in document["Metadata"]:
                creator = document["Metadata"]["/Creator"]
            if '/ModDate' in document["Metadata"]:
                last_modified = convertPyPDFDate(document["Metadata"]["/ModDate"])
            if '/Author' in document["Metadata"]:
                author = document["Metadata"]['/Author']        
        elif document['Domain']!= u'www.scouting.org':
            alternate_url = document['URL']
            alt_doc_name = document['File_Name']
            alt_primary_url = document['URL']
            alt_pages = document['Pages']
            
            if '/Title' in document["Metadata"]:
                alt_title = document["Metadata"]["/Title"]
            if '/CreationDate' in document["Metadata"]:
                alt_creation_date = convertPyPDFDate(document["Metadata"]["/CreationDate"])
            if '/Producer' in document["Metadata"]:
                alt_producer = document["Metadata"]["/Producer"]
            if '/Creator' in document["Metadata"]:
                alt_creator = document["Metadata"]["/Creator"]
            if '/ModDate' in document["Metadata"]:
                alt_last_modified = convertPyPDFDate(document["Metadata"]["/ModDate"])
            if '/Author' in document["Metadata"]:
                alt_author = document["Metadata"]['/Author']        
    
    if doc_name ==None and alt_doc_name!=None:
        doc_name = alt_doc_name
    
    if primary_url == None and alt_primary_url!=None:
        primary_url = alt_primary_url
        
    if pages == None and alt_pages!=None:
        pages=alt_pages
        
    if title == None and alt_title!=None:
        title = alt_title
        
    if creation_date == None and alt_creation_date!=None:
        creation_date = alt_creation_date
        
    if producer == None and alt_producer!=None:
        producer = alt_producer
        
    if creator == None and alt_creator!= None:
        creator = alt_creator
        
    if last_modified == None and alt_last_modified!=None:
        last_modified = alt_last_modified
        
    if author == None and alt_author!= None:
        author = alt_author
    
    return([content_hash,binary_hash,doc_name,primary_url,alternate_url,pages,
                 title,creation_date,producer,creator,last_modified,author])
#test return            
print(get_export_row("7297af00e1cc18bb961b799ff97028ea52044799"))    

[u'32c5c8557b32bef8a254d008b26afdac4fea21f6', '7297af00e1cc18bb961b799ff97028ea52044799', u'310-795-8_bookmark_4.pdf', u'http://www.scouting.org/filestore/marketing/stem/PDFs/310-795-8_bookmark_4.pdf', u'https://alumni.scouting.org/filestore/marketing/stem/PDFs/310-795-8_bookmark_4.pdf', 1, u'310-795-8.indd', u'2015-03-17', u'Adobe PDF Library 10.0.1', u'Adobe InDesign CS6 (Macintosh)', u'2015-03-17', None]


In [51]:
workbook = xlsxwriter.Workbook('BSA_PDF_Review.xlsx') #create new XLSX file
worksheet = workbook.add_worksheet() #create new worksheet in that XLSX file

worksheet.write_row(0,0,export_columns) #add name to first column

x=1
for bin_hash in working_hash_dict:
    worksheet.write_row(x,0,get_export_row(bin_hash))
    x+=1

workbook.close()

C:\Users\ajwil\Anaconda2\lib\site-packages\xlsxwriter\worksheet.py:829: UserWarning: Ignoring URL 'https://scoutnet.scouting.org/MyBSA/contentproxy1/proxy.aspx?Method=GetBinaryContent&Channel=InfoBinary&SessionToken=K1A5BDA8F4544CC18DA5292682436310C8BE80AC5D6D476199A7561125309420&Target=http%3A%2F%2Finfo.netbsa.org%2Fadm%2Fisd%2Fregistration%2Fpdf%2F524-925_2012.pdf' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
